In [ ]:
from cloudyfsps import outObj as ob
from cloudyfsps.outObj import getColors, nColors
from cloudyfsps.generalTools import calcQ
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rc('font', size=16, family='serif', serif=[r'cmr10'], style='normal', variant='normal', stretch='normal', weight='heavy')
mpl.rc('legend', labelspacing=0.1, handlelength=2, fontsize=16)
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

# Figure 5

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputCKC/'
mod_prefix='ZAU'
zau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True, read_cool=True)

In [ ]:
logZs, ages, logUs, fsps_Qs = np.genfromtxt('ckc_output_fsps_Q.txt', unpack=True)
for mod in zau.mods:
    ind, = np.where((mod.logU == logUs) & (mod.logZ == logZs) & (mod.age == ages))
    mod.fsps_Q = fsps_Qs[ind[0]]

In [ ]:
keys = ['frac_cool_CE', 'frac_cool_H']
#keys = ['log_cool_CE', 'log_cool_FFc', 'log_cool_H']
labs = [r'$\Lambda_{\mathrm{CE}}\;/\;\Lambda_{\mathrm{Total}}$',
        r'$\Lambda_{\mathrm{H}}\;/\;\Lambda_{\mathrm{Total}}$']
#cnames = ['Blues', 'Blues', 'Blues']
cb_arr = np.linspace(0.0,1.0)
sM, cNorm = getColors(cb_arr, cname='Blues', return_cNorm=True, maxv=0.95, minv=0.1)
ylabel=r'log Z/Z$_{\odot}$'
xlabel=r'log $\mathcal{U}_0$'

for key,lab in zip(keys,labs):
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    zau.pxl_plot(zval=key, clab=lab, ax=ax,
                 const='age', cval=1.e6, cmap=sM.cmap, sM=sM, cNorm=cNorm,
                 yval='logZ', ylab=ylabel,clab_size=16,
                 xval='logU', xlab=xlabel, calc_aspect=False)
    ax.set_ylabel(ylabel, size=16)
    ax.set_xlabel(xlabel, size=16)
    fig.subplots_adjust(left=0.15, bottom=0.125, top=0.95, right=0.95)
    #plt.tight_layout()
    fig.savefig('./f5.pdf', dpi=300)

# Figure 6

In [ ]:
def tplt(zau, ax, lw=2, col='k', logU=-2.5, logZ=-0.1, lab=None, key='logT0', alpha=1.0, **kwargs):
    modlist = [mod for mod in zau.mods if (mod.logU == logU) & (mod.logZ == logZ)]
    x = np.array([mod.age/1.e6 for mod in modlist])
    y = np.array([mod.__getattribute__(key) for mod in modlist])
    if lab is None:
        lab = r'${0:.1f}$'.format(logU)
    ax.plot(x,y, lw=lw, color=col, label=lab, alpha=alpha)
    return

In [ ]:
[mod.__setattr__('logT0', np.log10(mod.T0)) for mod in zau.mods]
zau.add_arrs('logT0')
key = 'logT0'
lab = r'Log Temperature (K)'
cname = 'gist_heat'
ylabel=r'log Z/Z$_{\odot}$'
xlabel=r'log $\mathcal{U}_0$'

fig, ax = plt.subplots(1, figsize=(5.,5.5))
pf = zau.pxl_plot(zval=key, clab=lab, ax=ax, no_cbar=True,
                  const='age', cval=1.e6, cname=cname,
                  yval='logZ', ylab=ylabel,
                  xval='logU', xlab=xlabel, calc_aspect=False)
ax.set_ylabel(ylabel, size=16)
ax.set_xlabel(xlabel, size=16)

top = 0.85
right = 0.97
left = 0.15
bottom=0.1
cbw=0.05
cbb = 0.01
fig.subplots_adjust(left=left, bottom=bottom, top=top, right=right)

Tlabs = [3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3]
cbar_ax = fig.add_axes([left, top+cbb, (right-left), cbw])
cb = fig.colorbar(pf, cax=cbar_ax, orientation='horizontal')
cb.set_label(lab, size=16)
cb.set_ticks(Tlabs)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
fig.savefig('./f6a.pdf', dpi=300)

In [ ]:
Zloop = [-0.3, -0.58, -0.98]
colors = nColors(len(Zloop), cname='plasma_r', minv=0.3, maxv=1.0)
alphas = (zau.logU_vals+5)/4.0
fig, ax2 = plt.subplots(1, figsize=(5.,5.))

lab = r'Temperature ($\times 10^{-3}$ K)'
key = 'T0'
for Z,ZCOL in zip(Zloop, colors):
    vlab = r'{0: .1f}'.format(Z)
    for U,UALPH in zip(zau.logU_vals, alphas):
        if U == -1.0:
            plab = vlab
        else:
            plab='__nolabel__'
        tplt(zau, ax2, logU=U, col=ZCOL, logZ=Z, key=key, alpha=UALPH, lab=plab)
leg = ax2.legend(loc=0, frameon=False, fontsize=16, title=r'log Z/Z$_{\odot}$')
plt.setp(leg.get_title(), fontsize=16)
ax2.set_ylabel(lab, size=16)
ax2.set_xlim(0.5,9.5)
ax2.set_ylim((10.**cb.get_clim()[0], 10.**cb.get_clim()[1]))
ax2.set_yticks([5.e3, 10.e3, 15.e3, 20.e3])
ax2.set_yticklabels([r'$5$',r'$10$',r'$15$',r'$20$'])
ax2.set_xlabel(r'Age [ Myr ]', size=16)
fig.subplots_adjust(left=left, bottom=bottom+0.02, top=0.98, right=right)
fig.savefig('./f6b.pdf', dpi=300)

# Figure 7

In [ ]:
[mod._init_emis() for mod in zau.mods];
for mod in zau.mods:
    mod.indHe = np.argmin(np.abs(mod.ion_arr['He'][1]-0.5))
    mod.indH = np.argmin(np.abs(mod.ion_arr['H'][1] - 0.5))
zau.add_arrs('indHe')
zau.add_arrs('indH')

In [ ]:
emis = dict(O=['O__3_500700A', 'O_II_372600A', 'O__1_630000A'],
            S=['S_II_671600A', 'S_II_673100A', 'S__3_953200A'],
            N=['N__2_658400A', 'N__2_654800A'],
            Ar=['AR_3_713500A'],
            Ne=['NE_3_386900A'],
            H=['H__1_656285A', 'H__1_486136A', 'H__1_434049A'],
            He=['HE_1_388863A', 'HE_1_447147A', 'HE_1_587561A'])
emis_labs=dict(O=[r'[O III]$\lambda$5007',
                  r'[O  II]$\lambda$3726',
                  r'[O   I]$\lambda$6300'],
               S=[r'\textbf{[S II]$\lambda$6716}',
                  r'\textbf{[S II]$\lambda$6731}',
                  r'\textbf{[S III]$\lambda$9532}'],
               N=[r'\textbf{[N II] $\lambda$6548}',
                  r'\textbf{[N II] $\lambda$6584}'],
               Ar=[r'\textbf{Ar 3 $\lambda$7135}'],
               Ne=[r'\textbf{Ne 3 $\lambda$3869}'],
               H=[r'$\mathrm{H}\alpha$', r'$\mathrm{H}\beta$', r'$\mathrm{H}\gamma$'],
               He=[r'\textbf{He I $\lambda$3889}',
                   r'\textbf{He I $\lambda$4471}',
                   r'\textbf{He I $\lambda$5876}'])

In [ ]:
def HHeIon(ax, x0, xH, xHe):
    ax.axvspan(x0, xH, color='black', alpha=0.07)
    ax.axvspan(x0, xHe, color='black', alpha=0.15)
    return ax
def tempPlt(ax, xarr, tarr, T0, lw=2, color='black', size=18, ylim=None):
    lab = r'$<\mathrm{{T}}_e> = {0:.0f}\,\mathrm{{K}}$'.format(T0)
    ax.plot(xarr, np.log10(tarr), lw=lw)
    ax.axhline(np.log10(T0), color=color, lw=lw, linestyle='--')
    ax.annotate(lab, xy=(0.6, 0.35),
                xytext=(0.6,0.35),
                xycoords='axes fraction', size=size)
    ax.set_ylabel(r'$\log$ [ T$_e$ (K) ]')
    if ylim is None:
        ax.set_ylim(0.96*np.log10(T0), np.max(np.log10(tarr))+0.06)
    else:
        ax.set_ylim(ylim)
    return ax
def ionPlt(ax, x, mod, species, fylim, lw=2, size=18, label=r'log [ X ]', colors=None, plot_leg=True):
    if colors is None:
        colors = ['red', 'green', 'blue']
    labs=[r'{0}'.format(species),
          r'{0}+'.format(species),
          r'{0}++'.format(species)]
    ax.plot(x, np.log10(mod.ion_arr[species][0]), lw=lw, color=colors[2], label=labs[0])
    ax.plot(x, np.log10(mod.ion_arr[species][1]), lw=lw, color=colors[1], label=labs[1])
    ax.plot(x, np.log10(mod.ion_arr[species][2]), lw=lw, color=colors[0], label=labs[2])
    crds = [[(x[mod.indH], (np.log10(mod.ion_arr[species][0])[-1])), (-20., -20.)],
            [(x[mod.indHe], np.log10(mod.ion_arr[species][1])[mod.indHe]-0.2), (10,-20)],
            [(x[0], np.log10(mod.ion_arr[species][2])[0]), (10,10)]]
    if (species == 'H') or (species == 'He'):
        crds = [[(x[mod.indH], (np.log10(mod.ion_arr[species][0])[-1])), (-20., -20.)],
                [(x[0], np.log10(mod.ion_arr[species][1])[0]), (10,-20)],
                [(x[0], np.log10(mod.ion_arr[species][2])[0]), (10,10)]]
    xlims = ax.get_xlim()
    for crd, lab in zip(crds, labs):
        if xlims[1] - crd[0][0] <= 0.1:
            xy = (crd[0][0]-0.01, crd[0][1])
        else:
            xy = crd[0]
        #ax.annotate(lab, xy=xy, xytext=crd[1],
        #            xycoords='data', textcoords='offset points',size=size)
    if plot_leg:
        ax.legend(loc=0, frameon=False)
    if label is not None:
        ax.set_ylabel(label, size=16)
    ax.set_ylim(fylim)
    return ax
def emisPlot(ax, x, mod, emis, emis_labs, plot_leg,
             bbox, eylim, lw=2, fontsize=14, colors=None):
    if colors is None:
        colors = ['red', 'green', 'blue']
    for em, lab, col in zip(emis, emis_labs, colors):
        ax.plot(x, np.log10(mod.get_emis(em)), lw=lw, label=lab, color=col)
    if plot_leg:
        if bbox is not None:
            ax.legend(loc=0, bbox_to_anchor=bbox, frameon=False, fontsize=fontsize)
        else:
            ax.legend(loc=0, frameon=False, fontsize=fontsize)
    ax.set_ylabel(r'\textbf{log Emissivity}')
    ax.set_ylim(eylim)
    return ax

In [ ]:
def cumEmis(mod, emis, emis_labs, ax=None, xunit='rad_pc', pl_leg=True, ylim=None,norm=True,
            xlab=r'Radius (pc)', ylab=r'Integrated Emissivity', labsize=16,
            lw=3, figsize=(8,6), fontsize=16, normTotal=True, shell=False, emisOnly=False,
            colors=None):
    if ax is None:
        fig, ax = plt.subplots(1, figsize=figsize)
    x = mod.__getattribute__(xunit)
    if colors is None:
        colors = ['blue', 'green', 'red']
    pc=False
    if xunit == 'rad_pc':
        pc=True
    outRad, outEms = get_interp_rad(mod, emis, norm=norm, pc=pc)
    for Em, lab, col in zip(outEms, emis_labs, colors):
        ax.semilogy(outRad, Em, label=lab, lw=lw, color=col)
    if pl_leg:
        ax.legend(loc=0, frameon=False, fontsize=fontsize)
    ax.set_xlabel(xlab, size=labsize)
    ax.set_ylabel(ylab, size=labsize)
    
    if ylim is None:
        ax.set_ylim(0, np.max(outEms))
    else:
        ax.set_ylim(ylim)
    return ax

In [ ]:
def get_interp_rad(mod, em_names, n=300, norm=True, pc=True):
    r_in, r_out = mod.r_in, mod.r_out
    r_edges = np.linspace(mod.r_in, mod.r_out, n)
    radius = np.array([(r_edges[i]+r_edges[i+1])*0.5 for i in range(len(r_edges)-1)])
    dr = np.diff(r_edges)
    dv = 4.*np.pi*radius**2*dr
    Ems = []
    for e in em_names:
        Ems.append(np.interp(radius, mod.radius_all, mod.get_emis(e)))
    if norm:
        output = [Em*dv/(Em*dv).sum() for Em in Ems]
    else:
        output = [Em*dv for Em in Ems]
    if pc:
        outrad = radius*3.24078e-19
    else:
        outrad = radius
    return outrad, output

In [ ]:
def panplot(mod, species='O', emis=['O__3_500700A', 'O_II_372600A', 'O__1_630000A'], lw=3,
            figsize=(6,5.5), fylim=(-2,0.3), tylim=None, norm=False,
            emis_labs=[r'[O III]$\lambda$5007',r'[O  II]$\lambda$3726',r'[O   I]$\lambda$6300'],
            xunit='depth', unitlab=r'Radius (pc)',
            eylim=(-22.5, -20.5), fontsize=22, plot_leg=True, bbox=None, buff=0.01, **kwargs):
    x = mod.__getattribute__(xunit)
    # EDITING HERE
    fig, (ax1, ax2) = plt.subplots(2, figsize=figsize, sharex=True)
    #
    HHeIon(ax1, x[0], x[mod.indH], x[mod.indHe])
    #tempPlt(ax1, x, mod.Te, mod.T0, lw=lw, size=fontsize, ylim=tylim)
    #
    #ax2 = plt.subplot(312, sharex=ax1)
    #HHeIon(ax2, x[0], x[mod.indH], x[mod.indHe])
    ionPlt(ax1, x, mod, species, fylim, lw=lw, size=fontsize+4, **kwargs)
    #
    #ax3 = plt.subplot(313, sharex=ax1)
    HHeIon(ax2, x[0], x[mod.indH], x[mod.indHe])
    #emisPlot(ax3, x, mod, emis, emis_labs, plot_leg, bbox,
    #         eylim, lw=lw, fontsize=fontsize, **kwargs)
    cumEmis(mod, emis, emis_labs, ysize=fontsize, fontsize=fontsize, pl_leg=plot_leg,
            xunit=xunit, xlab=unitlab, shell=True, norm=norm,
            colors=colors, ax=ax2, normTotal=True, lw=3, ylim=eylim)
    #ax3.set_xlabel(unitlab)
    #ax3.set_xlim(x[0]-buff, x[-1]+buff)
    fig.subplots_adjust(hspace=0)
    plt.setp(fig.axes[0].get_xticklabels(), visible=False)
    fig.subplots_adjust(left=0.12, right=0.97, top=0.98, bottom=0.1)#plt.tight_layout()
    return fig, (ax1, ax2)

In [ ]:
species='O'
emis=['O__3_500700A', 'O_II_372600A', 'O__1_630000A']
lw=3
figsize=(8,4)
fylim=(-1.7, 0.4)
tylim=(3.73, 3.97)
norm=False
emis_labs=[r'[O III]$\lambda$5007',r'[O  II]$\lambda$3726',r'[O   I]$\lambda$6300']
xunit='rad_pc'
unitlab=r'Radius (pc)'
eylim=(-0.01, 0.13)
fontsize=16
plot_leg=True
bbox=None
buff=0.05

colors = nColors(3, minv=0.4, maxv=0.9, cname='inferno_r')
fig, axes = plt.subplots(2,3, figsize=figsize, sharex='all', sharey='row')
for i, age in enumerate([1.0e6, 2.0e6, 3.0e6]):
    ind, = np.where((zau.logZ == 0.0) & (zau.age == age) & (zau.logU == -2.5))
    mod = zau.mods[ind]
    x = mod.__getattribute__(xunit)
    
    lab = r'${0:.0f}\;\mathrm{{Myr}}$'.format(mod.age/1.e6)
    lsz=24
    topax = axes[0,i]
    botax = axes[1,i]
    xlab=r'Radius (pc)'
    if i == 0:
        ylab = r'$\log[\,\mathcal{E} \cdot \mathrm{dV}\,]$'
        ilab = r'log [ O ]'
    else:
        ylab = ''
        ilab = ''
    if i == 2:
        plot_leg = True
    else:
        plot_leg = False
    HHeIon(topax, x[0], x[mod.indH], x[mod.indHe])
    ionPlt(topax, x, mod, species, fylim, lw=lw, label=ilab, size=fontsize+4, colors=colors, plot_leg=plot_leg)
    topax.set_xlim(3.25,3.93)
    cumEmis(mod, emis, emis_labs, fontsize=14, pl_leg=plot_leg,
            xunit=xunit, xlab=xlab, ylab=ylab, norm=False,
            colors=colors, ax=botax, normTotal=True, lw=3, ylim=(8.e32, 1.1e35))
    xx, yy = 0.03, 0.98
    topax.annotate(lab, xy=(xx,yy), xycoords="axes fraction",
                     size=lsz, horizontalalignment='left', va='top')

fig.subplots_adjust(hspace=0, wspace=0)
plt.setp([axes[0,i].get_xticklabels() for i in range(3)], visible=False)
plt.setp([axes[0,i].get_yticklabels() for i in range(1,3)], visible=False)
plt.setp([axes[1,i].get_yticklabels() for i in range(1,3)], visible=False)
fig.subplots_adjust(left=0.1, right=0.98, top=0.98, bottom=0.13)
fig.savefig('./f7.pdf', dpi=300)

# Figure 8

In [ ]:
species='O'
emis=['O__3_500700A', 'O_II_372600A', 'O__1_630000A']
lw=3
figsize=(8,4)
fylim=(-1.7, 0.4)
tylim=(3.73, 3.97)
norm=False
emis_labs=[r'[O III]$\lambda$5007',r'[O  II]$\lambda$3726',r'[O   I]$\lambda$6300']
xunit='rad_pc'
unitlab=r'Radius (pc)'
eylim=(-0.01, 0.13)
fontsize=16
plot_leg=True
bbox=None
buff=0.05

colors = nColors(3, minv=0.4, maxv=0.9, cname='inferno_r')
fig, axes = plt.subplots(2,3, figsize=figsize, sharex='col', sharey='row')
for i,logZ in enumerate([-0.98, -0.3, 0.0]):
    ind, = np.where((zau.logZ == logZ) & (zau.age == 3.0e6) & (zau.logU == -2.5))
    mod = zau.mods[ind]
    lab = r'$\log\,\mathrm{{Z}}/\mathrm{{Z}}_{{\odot}} = {0:.1f}$'.format(mod.logZ)
    x = mod.__getattribute__(xunit)
    xlim = (np.min(x), np.max(x))
    topax = axes[0,i]
    botax = axes[1,i]
    xlab=r'Radius (pc)'
    if i == 0:
        ylab = r'$\log[\,\mathcal{E} \cdot \mathrm{dV}\,]$'
        ilab = r'log [ O ]'
    else:
        ylab = ''
        ilab = ''
    if i == 1:
        plot_leg = True
    else:
        plot_leg = False
    HHeIon(topax, x[0], x[mod.indH], x[mod.indHe])
    ionPlt(topax, x, mod, species, fylim, lw=lw, label=ilab, size=fontsize+4, colors=colors, plot_leg=plot_leg)
    topax.set_xlim(xlim)
    cumEmis(mod, emis, emis_labs, fontsize=14, pl_leg=plot_leg,
            xunit=xunit, xlab=xlab, ylab=ylab, norm=False,
            colors=colors, ax=botax, normTotal=True, lw=3, ylim=(6.e31, 6.e34))
    xx, yy = 0.03, 0.98
    topax.annotate(lab, xy=(xx,yy), xycoords="axes fraction",
                     size=18, horizontalalignment='left', va='top')
    if i == 0:
        botax.set_xticks([3.4, 3.6, 3.8, 4.0, 4.2])
    if i == 1:
        botax.set_xticks([3.4, 3.6, 3.8, 4.0])
    if i == 2:
        botax.set_xticks([3.4, 3.6, 3.8])
fig.subplots_adjust(hspace=0, wspace=0)
plt.setp([axes[0,i].get_xticklabels() for i in range(3)], visible=False)
plt.setp([axes[0,i].get_yticklabels() for i in range(1,3)], visible=False)
plt.setp([axes[1,i].get_yticklabels() for i in range(1,3)], visible=False)
fig.subplots_adjust(left=0.1, right=0.98, top=0.98, bottom=0.13)
fig.savefig('./finalFigs/f8.pdf', dpi=300)

# Figure 9

In [ ]:
#[mod.__setattr__('log_Ha_Q', 11.875+np.log10(mod.Ha/(10.**mod.logQ))) for mod in zau.mods]
#lab = r'12 + log[ $\mathrm{L}(\mathrm{H}\alpha)\;/\;Q_{\mathrm{H}}$ ]'
[mod.__setattr__('log_Ha_Q', np.log10(mod.Ha/(10.**mod.logQ))) for mod in zau.mods]
lab = r'log[ $\mathrm{L}(\mathrm{H}\alpha)\;/\;Q_{\mathrm{H}}$ ]'
zau.add_arrs('log_Ha_Q')
key = 'log_Ha_Q'


Zloop = [0.0, -0.3, -0.58, -0.98, -1.98]
colors = nColors(len(Zloop), cname='plasma_r', minv=0.3, maxv=1.0)
alphas = (zau.logU_vals+5)/4.0

fig, ax2 = plt.subplots(1, figsize=(6.5,5.5))

for Z,ZCOL in zip(Zloop, colors):
    vlab = r'{0: .1f}'.format(Z)
    for U,UALPH in zip(zau.logU_vals, alphas):
        if U == -1.0:
            plab = vlab
        else:
            plab='__nolabel__'
        tplt(zau, ax2, logU=U, col=ZCOL, logZ=Z, key=key, alpha=UALPH, lab=plab)
leg = ax2.legend(loc=0, frameon=False, fontsize=16, title=r'log Z/Z$_{\odot}$')
plt.setp(leg.get_title(), fontsize=16)
ax2.set_ylabel(lab, size=16)
ax2.set_xlabel(r'Age [ Myr ]', size=16)
ax2.set_xlim(0.5, 9.5)
ax2.set_ylim(-11.875,-11.71)
#ax2.set_ylim(0.0,0.16)
ax2.tick_params(axis='y', labelsize=11)
fig.subplots_adjust(left=0.12, bottom=0.12, top=0.98, right=0.98)
fig.savefig('./f9.pdf', dpi=300)

# Figure 10

In [ ]:
[mod._load_cont(output_units=True) for mod in zau.mods];

In [ ]:
def add_lim(ax, lam=911.8, add_text=True, txt=r'$\mathrm{Ly}$', color='k', lw=2.0,
            linestyle='dashed', xbuff=0.1, ybuff=0.1, fontsize=12):
    ax.axvline(np.log10(lam), lw=lw, linestyle=linestyle, color=color)
    if add_text:
        ax.text(np.log10(lam)+xbuff, ax.get_ylim()[0]+ybuff, txt, fontsize=fontsize)
    return ax

In [ ]:
zinds = np.where((zau.age == 1.0e6) & (zau.logZ == 0.0))[0]
zmods = [zau.mods[z] for z in zinds]
[mod.get_fsps_spec() for mod in zmods]
sM = getColors(zau.logU_vals)
fsps_Q = np.log10(zmods[0].fsps_Q)
[mod.logU for mod in zmods]

ainds = np.where((zau.age == 3.0e6) & (zau.logZ == 0.0))[0]
amods = [zau.mods[z] for z in ainds]
[mod.get_fsps_spec() for mod in amods]
sM = getColors(zau.logU_vals)
afsps_Q = np.log10(amods[0].fsps_Q)
[mod.logU for mod in amods]

In [ ]:
xarrs_1Myr=[np.log10(mod.lam) for mod in zmods]
yarrs_1Myr=[np.log10(mod.nebflu)+fsps_Q-np.log10(mod.Qh) for mod in zmods]
logU_1Myr = [mod.logU for mod in zmods]

xarrs_3Myr=[np.log10(mod.lam) for mod in amods]
yarrs_3Myr=[np.log10(mod.nebflu)+afsps_Q-np.log10(mod.Qh) for mod in amods]
logU_3Myr = [mod.logU for mod in amods]

sMu = getColors(zau.logU_vals)

In [ ]:
zs = zau.logZ_vals[2:-1]
ZQs = []
for i,zz in enumerate(zs):
    zinds = np.where((zau.age == 3.0e6) & (zau.logZ == zz))[0]
    zmods = [zau.mods[z] for z in zinds]
    zmods[0].get_fsps_spec()
    fsps_Q = np.log10(zmods[0].fsps_Q)
    ZQs.append(fsps_Q)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, (ax, ax2) = plt.subplots(2, sharex=True, sharey=True, figsize=(10,6.5))
ylim = (-15.1,-12.7)
xlim = (2.95,5.01)

for x1,y1,u1,x3,y3,u3 in zip(xarrs_1Myr, yarrs_1Myr, logU_1Myr, xarrs_3Myr, yarrs_3Myr, logU_3Myr):
    ax.plot(x1, y1, color=sMu.to_rgba(u1), lw=3)
    ax.plot(x3, y3, color=sMu.to_rgba(u3), lw=3)

ax.annotate(r'1 Myr', xy=(0.06, 0.74), xycoords='axes fraction', fontsize=18)
ax.annotate(r'3 Myr', xy=(0.2, 0.42), xycoords='axes fraction', fontsize=18)
ax.annotate(r'Z$_{\odot}$', xy=(0.98, 0.05), xycoords='axes fraction', fontsize=24, ha='right')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", "3%", pad="1%")
cb = fig.colorbar(sMu, cax=cax)
cb.set_label(r'$\log\;\mathcal{U}_0$', fontsize=18)
cb.set_ticks([-4,-3,-2,-1])
ax.set_ylim(ylim)
ax.set_xlim(xlim)
ax.set_yticks([-15, -14, -13])

add_lim(ax, lam=911.8, txt=r'H(Ly)', fontsize=14, ybuff=0.07, xbuff=0.01)
add_lim(ax, lam=3646., txt=r'H(Ba)', fontsize=14, ybuff=0.07, xbuff=0.01)
add_lim(ax, lam=8204.0, txt=r'H(Pa)', fontsize=14, ybuff=0.07, xbuff=0.01)
add_lim(ax, lam=14580.0, txt=r'H(Br)', fontsize=14, ybuff=0.07, xbuff=0.01)

temps = np.array([mod.T0 for mod in zau.mods if ((mod.age == 3.e6)
                                                 & (mod.logZ <0.2)
                                                 & (mod.logZ > -1.0)
                                                 & (mod.logU == -1.))])
ss = getColors(np.log10(temps), cname='gist_heat')

for mod in zau.mods:
    if (mod.age == 3.e6) & (mod.logU == -1.0) & (mod.logZ <0.2) & (mod.logZ >-1.0):
        ind, = np.where((zs == mod.logZ))
        if int(ind) < 9:
            fsps_Q = ZQs[int(ind)]
            col = ss.to_rgba(np.log10(mod.T0))
            ax2.plot(np.log10(mod.lam), np.log10(mod.nebflu)+fsps_Q-np.log10(mod.Qh), color=col, lw=3)
            
ax2.annotate(r'3 Myr', xy=(0.2, 0.34), xycoords='axes fraction', fontsize=18)

add_lim(ax2, lam=911.8, add_text=False, ybuff=0.07, xbuff=0.01)
add_lim(ax2, lam=3646., add_text=False, ybuff=0.07, xbuff=0.01)
add_lim(ax2, lam=8204.0, add_text=False, ybuff=0.07, xbuff=0.01)
add_lim(ax2, lam=14580.0, add_text=False, ybuff=0.07, xbuff=0.01)

divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", "3%", pad="1%")
cb = fig.colorbar(ss, cax=cax2)
cb.set_label(r'$\log\;T_{\mathrm{eff}}~[ \mathrm{K} ]$', fontsize=18)
cb.set_ticks([3.8, 4.0, 4.2])
fig.text(0.05, 0.5,
         r'$\log\,\mathrm{{L}}_{{\nu}} \cdot \hat{Q}_{\mathrm{H}} / Q_{\mathrm{H}}$',
         size=18, ha='center', va='center', rotation='vertical')
ax2.set_xlabel(r'$\log\mathrm{{Wavelength}}$ [$\mathrm{\AA}$]', size=18)

fig.subplots_adjust(hspace=0.05, right=0.93, left=0.1, top=0.98, bottom=0.1)
plt.setp(fig.axes[0].get_xticklabels(), visible=False)
fig.savefig('./f10.pdf', dpi=300)

# Figure 11

In [ ]:
def frac_flux(ages, starfrac, contfrac, linefrac,
              ax=None, ageind = range(0,51), set_axlims=False, xlims=(5.5, 7.7), ylims=(0.0,1.0),
              set_axlabels=False, xlab=r'\textbf{log t (years)}', ylab=r'\textbf{Fractional Flux [4000-10,000\AA]}',
              size=18, colors=['#fbb4ae','#b3cde3','#ccebc5'], alpha=1.0, lw=2, mnorm=None):
    if ax is None:
        fig, ax = plt.subplots(1, figsize=(8,6))
    ax.fill_between(ages[ageind], 0, y2=starfrac[ageind],
                    color=colors[0], alpha=alpha)
    ax.plot(ages[ageind],
            starfrac[ageind],
            lw=2, color='k')
    ax.fill_between(ages[ageind], starfrac[ageind],
                    y2=starfrac[ageind]+contfrac[ageind],
                    color=colors[1], alpha=alpha)
    ax.plot(ages[ageind],
            starfrac[ageind]+contfrac[ageind],
            lw=2, color='k')
    ax.fill_between(ages[ageind], starfrac[ageind]+contfrac[ageind],
                    y2=starfrac[ageind]+contfrac[ageind]+linefrac[ageind],
                    color=colors[2], alpha=alpha)
    ax.plot(ages[ageind],
            starfrac[ageind]+contfrac[ageind]+linefrac[ageind],
            lw=2, color='k')
    if set_axlims:
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
    if set_axlabels:
        ax.set_xlabel(xlab, size=size)
        ax.set_ylabel(ylab, size=size)
    return

def calc_fracflux(wav, sp_star, sp_line, sp_all, wavinds=None, wavlims=None, mnorm=None):
    if wavinds is None:
        if wavlims is None:
            wavinds = (wav > 3800.) & (wav < 10000.)
        else:
            wavinds = (wav >= wavlims[0]) & (wav <= wavlims[1])
    starflux = sp_star[:, wavinds]
    lineflux = sp_line[:, wavinds] - sp_star[:, wavinds]
    linecontflux = sp_all[:, wavinds] - sp_star[:, wavinds]
    contflux = linecontflux - lineflux
    totflux = sp_all[:, wavinds]
    
    starfrac = np.sum(starflux, axis=1)/np.sum(totflux, axis=1)
    contfrac = np.sum(contflux, axis=1)/np.sum(totflux, axis=1)
    linefrac = np.sum(lineflux, axis=1)/np.sum(totflux, axis=1)
    return (starfrac, contfrac, linefrac)
def calc_fracspec(wav, sp_star, sp_line, sp_all, wavinds=None, wavlims=None, mnorm=None):
    if wavinds is None:
        if wavlims is None:
            wavinds = (wav > 3800.) & (wav < 10000.)
        else:
            wavinds = (wav >= wavlims[0]) & (wav <= wavlims[1])
    starflux = sp_star[:, wavinds]
    lineflux = sp_line[:, wavinds] - sp_star[:, wavinds]
    linecontflux = sp_all[:, wavinds] - sp_star[:, wavinds]
    contflux = linecontflux - lineflux
    totflux = sp_all[:, wavinds]
    
    starfrac = starflux
    contfrac = contflux
    linefrac = lineflux
    return (wav[wavinds], starfrac, contfrac, linefrac)

In [ ]:
wavlim = (3000, 10000)
####
spBC = fsps.StellarPopulation(zcontinuous=True, add_neb_emission=False)
spBC.params['logzsol'] = 0.0
spBC.params['gas_logz'] = 0.0
lam, specsBC = spBC.get_spectrum(peraa=True)
#
spAD = fsps.StellarPopulation(zcontinuous=True, add_neb_emission=True)
spAD.params['logzsol'] = 0.0
spAD.params['gas_logz'] = 0.0
lam, specsAD = spAD.get_spectrum(peraa=True)
#
spNC= fsps.StellarPopulation(zcontinuous=True, add_neb_emission=True, add_neb_continuum=False)
spNC.params['logzsol'] = 0.0
spNC.params['gas_logz'] = 0.0
lam, specsNC = spNC.get_spectrum(peraa=True)
#
wav, starfrac, contfrac, linefrac = calc_fracspec(spAD.wavelengths, specsBC, specsNC, specsAD, wavlims=wavlim)
######
spBC = fsps.StellarPopulation(zcontinuous=True, add_neb_emission=False)
spBC.params['logzsol'] = -1.0
spBC.params['gas_logz'] = -1.0
lam, specsBC = spBC.get_spectrum(peraa=True)
#
spAD = fsps.StellarPopulation(zcontinuous=True, add_neb_emission=True)
spAD.params['logzsol'] = -1.0
spAD.params['gas_logz'] = -1.0
lam, specsAD = spAD.get_spectrum(peraa=True)
#
spNC= fsps.StellarPopulation(zcontinuous=True, add_neb_emission=True, add_neb_continuum=False)
spNC.params['logzsol'] = -1.0
spNC.params['gas_logz'] = -1.0
lam, specsNC = spNC.get_spectrum(peraa=True)
#
wav, starfracz, contfracz, linefracz = calc_fracspec(spAD.wavelengths, specsBC, specsNC, specsAD, wavlims=wavlim)

In [ ]:
ageinds = range(0,31)
ages = 10.**spAD.ssp_ages[ageinds]/1.e6
ageind_list = [10, 20]#24 is 5myr
labs = [r'1 Myr', r'3 Myr']
fig, axes = plt.subplots(1,2, figsize=(10.,3.5), sharey='row', sharex='all')

colors = nColors(2, minv=0.05, maxv=0.95, cname='RdBu')
colors2 = nColors(2, minv=0.1, maxv=0.9, cname='RdBu')

yl = (0.2e-3, 3.0)
for ax, ageind in zip(fig.axes, ageind_list):
    ax.semilogy(wav, starfrac[ageind,:]+linefrac[ageind,:]+contfrac[ageind,:],
                color=colors2[0], lw=2, alpha=0.8)
    ax.semilogy(wav, starfrac[ageind,:], color=colors[0], lw=2, alpha=1.0, label=r'0.0')
    ax.semilogy(wav, starfracz[ageind,:]+linefracz[ageind,:]+contfracz[ageind,:],
                color=colors2[1], lw=2, alpha=0.8)
    ax.semilogy(wav, starfracz[ageind,:], color=colors[1], lw=2, alpha=1.0, label=r'-1.0')
    ax.set_xlim(3050,9950)
    ax.set_ylim(yl)
    ax.annotate(r'{0:.0f} Myr'.format(ages[ageind]),
                xy=(0.85, 0.9), xycoords='axes fraction', ha='right', size=28)
fig.subplots_adjust(hspace=0, wspace=0, right=0.99, top=0.98, left=0.08, bottom=0.16)

leg = fig.axes[0].legend(loc=3, title=r'log Z/Z$_{\odot}$', fontsize=16, frameon=False)
plt.setp(leg.get_title(), fontsize=16)

leg = fig.axes[1].legend(loc=3, title=r'log Z/Z$_{\odot}$', fontsize=16, frameon=False)
plt.setp(leg.get_title(), fontsize=16)
plt.setp(fig.axes[1].get_yticklabels(), visible=False)

[a.set_xlabel(r'Wavelength [$\,\mathrm{\AA}\,$]', size=16) for a in fig.axes]
[fig.axes[i].set_ylabel(r'$\mathcal{F}_{\lambda}\ [\,\mathrm{L}_{\odot}/\mathrm{\AA}\,]$', size=16) for i in [0]]
fig.savefig('./f11.pdf', dpi=300)

# Figure 12

In [ ]:
Zit = [0.0, -1.0]
wavlim_list = [(1340,1806), (1693, 3007), (3863, 5562), (6884,9659), (31296, 39614), (39173, 50561)]
ylabs = [r'FUV',
         r'NUV',
         r'F475W',
         r'F814W',
         r'3.6 $\mu$m',
         r'4.5 $\mu$m']
wavlim_list = [(900,1800.), (1800,4000.), (4000, 10000.), (10000.,27000.), (27000, 53000.)]
ylabs = [r'900-1800$\mathrm{\AA}$',
         r'1800-4000$\mathrm{\AA}$',
         r'4000-10000$\mathrm{\AA}$',
         r'1.0 - 2.7 $\mu$m',
         r'2.7 - 5.3 $\mu$m']

fig, axes = plt.subplots(2,5, figsize=(10,5))
for row, Z in enumerate(Zit):
    ######
    spBC = fsps.StellarPopulation(zcontinuous=True, add_neb_emission=False)
    spBC.params['logzsol'] = Z
    spBC.params['gas_logz'] = Z
    lam, specsBC = spBC.get_spectrum(peraa=True)
    #
    spAD = fsps.StellarPopulation(zcontinuous=True, add_neb_emission=True)
    spAD.params['logzsol'] = Z
    spAD.params['gas_logz'] = Z
    lam, specsAD = spAD.get_spectrum(peraa=True)
    #
    spNC= fsps.StellarPopulation(zcontinuous=True, add_neb_emission=True, add_neb_continuum=False)
    spNC.params['logzsol'] = Z
    spNC.params['gas_logz'] = Z
    lam, specsNC = spNC.get_spectrum(peraa=True)
    #
    wav, starfracz, contfracz, linefracz = calc_fracspec(spAD.wavelengths, specsBC, specsNC, specsAD, wavlims=wavlim)
    for wavlim, ylab, ax in zip(wavlim_list, ylabs, axes[row,:]):
        starfrac, contfrac, linefrac = calc_fracflux(spAD.wavelengths, specsBC, specsNC, specsAD, wavlims=wavlim)
        frac_flux(spAD.ssp_ages, starfrac, contfrac, linefrac,
                  ax=ax, set_axlims=True, set_axlabels=False, ylab=ylab, size=14)
        ax.annotate(ylab, fontsize=16, xy=(0.98, 0.02),
                    xycoords='axes fraction', ha='right', va='bottom')
        if ylab != ylabs[0]:
            plt.setp(ax.get_yticklabels(), visible=False)
        else:
            ax.set_ylabel(r'Fractional Flux', size=14)
            ax.set_yticks([0.2,0.4,0.6,0.8])
            zlab=r'log Z/Z$_{{\odot}}$={0:.1f}'.format(Z)
            ax.annotate(zlab, xy=(0.02, 0.4), xycoords='axes fraction',
                        size=16, ha='left', va='bottom')
        if row == 0:
            plt.setp(ax.get_xticklabels(), visible=False)
        if row == 1:
            ax.set_xlabel(r'log Age [ years ]', size=14)
            ax.set_xticks([6.0,6.5,7.0,7.5])
fig.axes[2].annotate(r'Line',
                     xy=(0.05,0.7), xycoords='axes fraction',
                     size=14, ha='left')
fig.axes[2].annotate(r'Cont',
                     xy=(0.05,0.4), xycoords='axes fraction',
                     size=14, ha='left')
fig.axes[2].annotate(r'Star',
                     xy=(0.05,0.1), xycoords='axes fraction',
                     size=14, ha='left')
fig.subplots_adjust(hspace=0, wspace=0, right=0.99, top=0.88, left=0.06, bottom=0.12)

for ax in axes[0,:]:
    ax2 = ax.twiny()
    ax2.set_xlabel(r'Age [ Myr ]', size=14)

    desired_tick_locations = np.array([1, 3, 5, 10])
    def tick_function(X):
        V = np.log10(X * 1.e6)
        return V
    tick_locations = tick_function(desired_tick_locations)
    def tick_labels(V):
        return [r"${0:.0f}$".format(z) for z in V]
    ax2.set_xlim(ax.get_xlim())
    ax2.set_xticks(tick_locations)
    ax2.set_xticklabels(tick_labels(desired_tick_locations))
fig.savefig('/f12.pdf', dpi=300)

In [ ]:
Zit = [0.0, -1.0]
wavlim_list = [(1340,1806), (1693, 3007), (3863, 5562), (6884,9659), (31296, 39614), (39173, 50561)]
ylabs = [r'FUV',
         r'NUV',
         r'F475W',
         r'F814W',
         r'3.6 $\mu$m',
         r'4.5 $\mu$m']
wavlim_list = [(900,1800.), (1800,4000.), (4000, 10000.), (10000.,27000.), (27000, 53000.)]
ylabs = [r'900-1800$\mathrm{\AA}$',
         r'1800-4000$\mathrm{\AA}$',
         r'4000-10000$\mathrm{\AA}$',
         r'1.0 - 2.7 $\mu$m',
         r'2.7 - 5.3 $\mu$m']

fig, axes = plt.subplots(2,5, figsize=(10,5))
for row, Z in enumerate(Zit):
    ######
    spBC = fsps.StellarPopulation(zcontinuous=True,
                                  add_neb_emission=False,
                                  sfh=1, const=1.0)
    spBC.params['logzsol'] = Z
    spBC.params['gas_logz'] = Z
    lam, specsBC = spBC.get_spectrum(peraa=True)
    #
    spAD = fsps.StellarPopulation(zcontinuous=True,
                                  add_neb_emission=True,
                                  sfh=1, const=1.0)
    spAD.params['logzsol'] = Z
    spAD.params['gas_logz'] = Z
    lam, specsAD = spAD.get_spectrum(peraa=True)
    #
    spNC= fsps.StellarPopulation(zcontinuous=True,
                                 add_neb_emission=True,
                                 add_neb_continuum=False, sfh=1, const=1.0)
    spNC.params['logzsol'] = Z
    spNC.params['gas_logz'] = Z
    lam, specsNC = spNC.get_spectrum(peraa=True)
    #
    wav, starfracz, contfracz, linefracz = calc_fracspec(spAD.wavelengths,
                                                         specsBC, specsNC, specsAD,
                                                         wavlims=wavlim)
    for wavlim, ylab, ax in zip(wavlim_list, ylabs, axes[row,:]):
        starfrac, contfrac, linefrac = calc_fracflux(spAD.wavelengths,
                                                     specsBC, specsNC, specsAD,
                                                     wavlims=wavlim)
        frac_flux(spAD.ssp_ages, starfrac, contfrac, linefrac,
                  ax=ax, set_axlims=True, set_axlabels=False,
                  ylab=ylab, size=14,
                  ageind=range(0,72), xlims=(5.5,9.1))
        ax.annotate(ylab, fontsize=16, xy=(0.98, 0.02),
                    xycoords='axes fraction', ha='right', va='bottom')
        if ylab != ylabs[0]:
            plt.setp(ax.get_yticklabels(), visible=False)
        else:
            ax.set_ylabel(r'Fractional Flux', size=14)
            ax.set_yticks([0.2,0.4,0.6,0.8])
            zlab=r'log Z/Z$_{{\odot}}$={0:.1f}'.format(Z)
            ax.annotate(zlab, xy=(0.02, 0.4), xycoords='axes fraction',
                        size=16, ha='left', va='bottom')
        if row == 0:
            plt.setp(ax.get_xticklabels(), visible=False)
        if row == 1:
            ax.set_xlabel(r'log Age [ years ]', size=14)
            ax.set_xticks([6.0,7.0,8.0, 9.0])
fig.axes[2].annotate(r'Line',
                     xy=(0.03,0.7), xycoords='axes fraction',
                     size=14, ha='left')
fig.axes[2].annotate(r'Cont',
                     xy=(0.03,0.35), xycoords='axes fraction',
                     size=14, ha='left')
fig.axes[2].annotate(r'Star',
                     xy=(0.03,0.1), xycoords='axes fraction',
                     size=14, ha='left')
fig.subplots_adjust(hspace=0, wspace=0, right=0.99, top=0.88, left=0.06, bottom=0.12)

for ax in axes[0,:]:
    ax2 = ax.twiny()
    ax2.set_xlabel(r'Age [ Myr ]', size=14)

    desired_tick_locations = np.array([1, 10, 100, 1000.])
    def tick_function(X):
        V = np.log10(X * 1.e6)
        return V
    tick_locations = tick_function(desired_tick_locations)
    def tick_labels(V):
        return [r"${0:.0f}$".format(z) for z in V]
    ax2.set_xlim(ax.get_xlim())
    ax2.set_xticks(tick_locations)
    ax2.set_xticklabels(tick_labels(desired_tick_locations))
fig.savefig('./f12b.png', dpi=300)

# Figure 13

In [ ]:
[mod.__setattr__('HbQNorm', np.log10(mod.Hb/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('HaQNorm', np.log10(mod.Ha/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('O3QNorm', np.log10(mod.OIIIb/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('S2QNorm', np.log10(mod.SIIb/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('N2QNorm', np.log10(mod.NIIb/mod.Qh)) for mod in zau.mods];

In [ ]:
[mod.__setattr__('HbNorm', np.log10(mod.Hb*mod.fsps_Q/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('HaNorm', np.log10(mod.Ha*mod.fsps_Q/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('O3Norm', np.log10(mod.OIIIb*mod.fsps_Q/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('S2Norm', np.log10(mod.SIIb*mod.fsps_Q/mod.Qh)) for mod in zau.mods];
[mod.__setattr__('N2Norm', np.log10(mod.NIIb*mod.fsps_Q/mod.Qh)) for mod in zau.mods];

In [ ]:
linelabs = [r'H$\beta$', r'[O III]',
            r'H$\alpha$', r'[N II]', r'[S II]']
wavelabs = [r'\textbf{4861\AA}', r'\textbf{5007\AA}',
            r'\textbf{6563\AA}', r'\textbf{6584\AA}', r'\textbf{6713\AA}']
keys = ['HbNorm','O3Norm','HaNorm','N2Norm','S2Norm']
Zloop = [0.0, -0.2, -0.39, -0.98]
colors = nColors(len(Zloop), cname='plasma_r', minv=0.3, maxv=1.0)
alphas = (zau.logU_vals+5)/4.0

fig, axes = plt.subplots(1,len(linelabs), figsize=(8.5,2.25))

for i, ax in enumerate(fig.axes):
    for logZ,ZCOL in zip(Zloop, colors):
        vlab = r'{0: .1f}'.format(logZ)
        for U,UALPH in zip(zau.logU_vals, alphas):
            if U == -1.0:
                plab = vlab
            else:
                plab='__nolabel__'
            tplt(zau, ax, logU=U, col=ZCOL, logZ=logZ, key=keys[i], alpha=UALPH, lab=plab)
    if i == 2:
        leg = ax.legend(loc=3, frameon=False, fontsize=12, title=r'log Z/Z$_{\odot}$')
        plt.setp(leg.get_title(), fontsize=14)
    ax.set_xlabel(r'Age (Myr)', size=16)
    ax.set_xlim(0.5, 9.5)
    ax.annotate(linelabs[i], xy=(0.98,0.98), xycoords='axes fraction',
                ha='right', va='top', size=22)
    ax.set_ylim(30.75, 35.25)

fig.axes[0].set_ylabel(r'$\log[\;\mathrm{{L}} \cdot \hat{Q}_{\mathrm{H}} / Q_{\mathrm{H}}\;]$', size=16)
plt.setp([a.get_yticklabels() for a in fig.axes[1::]], visible=False)
fig.subplots_adjust(left=0.08, bottom=0.22, top=0.98, right=0.98, wspace=0)
fig.savefig('./f13.pdf', dpi=300)

# Figure 14

In [ ]:
keys = ['log_OIII_Hb', 'log_OIII_OII', 'log_NII_Ha']
sls = ['a', 'b', 'c']
labs = [r'[$\,$O III$\,$]$\lambda$5007 / H$\beta$',
        r'[$\,$O III$\,$]$\lambda$5007 / [O II]$\lambda$3729',
        r'[$\,$N II$\,$]$\lambda$6584 / H$\alpha$']

ylabel=r'log Z/Z$_{\odot}$'
xlabel=r'log $\mathcal{U}_0$'

for key,lab, sl in zip(keys,labs,sls):
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    zau.pxl_plot(zval=key, clab=lab, ax=ax,
                 const='age', cval=1.e6, cname='inferno',
                 yval='logZ', ylab=ylabel,clab_size=16,
                 xval='logU', xlab=xlabel, calc_aspect=False)
    ax.set_ylabel(ylabel, size=16)
    ax.set_xlabel(xlabel, size=16)
    fig.subplots_adjust(left=0.15, bottom=0.125, top=0.95, right=0.95)
    fig.savefig('./f14{}.pdf'.format(sl), dpi=300)